# Download and Clean Dataset


In [1]:
import pandas as pd
import numpy as np

## Import data from source and explore it

In [2]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cement cubic meter, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. the first concrete sample is much closer to the second concrete sample

In [3]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

### Split data into predictors and target

In [5]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Check of the predictors and the target dataframes.

In [6]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [7]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Normalize the data by substracting the mean and dividing by the standard deviation.

In [8]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [9]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

## Import Keras

In [10]:
import keras

Using TensorFlow backend.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [11]:
from keras.models import Sequential
from keras.layers import Dense

Build a Neural Network that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

In [12]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

 Randomly split the data into a training and test sets

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)


# Train and Test the Network

In [14]:
# call the model
model = regression_model()
# fit the model with 100 epochs
epochs = 100
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/100
721/721 [==============================] - 0s 126us/step - loss: 1623.8882
Epoch 2/100
721/721 [==============================] - 0s 29us/step - loss: 1610.0545
Epoch 3/100
721/721 [==============================] - 0s 25us/step - loss: 1597.0177
Epoch 4/100
721/721 [==============================] - 0s 32us/step - loss: 1584.6970
Epoch 5/100
721/721 [==============================] - 0s 35us/step - loss: 1573.1045
Epoch 6/100
721/721 [==============================] - 0s 28us/step - loss: 1561.6250
Epoch 7/100
721/721 [==============================] - 0s 31us/step - loss: 1550.4820
Epoch 8/100
721/721 [==============================] - 0s 27us/step - loss: 1538.7900
Epoch 9/100
721/721 [==============================] - 0s 31us/step - loss: 1526.9021
Epoch 10/100
721/721 [==============================] - 0s 33us/step - loss: 1514.7086
Epoch 11/100
721/721 [==============================] - 0s 30us/step - loss: 1501.6179
Epoch 12/100
721/721 [=============================

Epoch 96/100
721/721 [==============================] - 0s 31us/step - loss: 175.4944
Epoch 97/100
721/721 [==============================] - 0s 33us/step - loss: 173.4722
Epoch 98/100
721/721 [==============================] - 0s 30us/step - loss: 171.5876
Epoch 99/100
721/721 [==============================] - 0s 32us/step - loss: 169.7308
Epoch 100/100
721/721 [==============================] - 0s 30us/step - loss: 167.9520


## Evaluating the model on the test data

In [15]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 53us/step


166.75299264852282

##  Compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [16]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

166.7529928161886 0.0


### Create a list of 50 mean squared errors.

In [17]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("The mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 111.63702506154873
MSE 2: 92.33397802951652
MSE 3: 57.2709170468031
MSE 4: 47.37973033571706
MSE 5: 45.66597683607182
MSE 6: 48.59738686323937
MSE 7: 45.8911518726534
MSE 8: 33.4150715676712
MSE 9: 36.716163931541075
MSE 10: 33.04954454736802
MSE 11: 35.75011877411777
MSE 12: 33.16249807907154
MSE 13: 36.79309894043265
MSE 14: 40.71011647591699
MSE 15: 32.34224412665012
MSE 16: 27.868711403658477
MSE 17: 30.950587578190184
MSE 18: 32.60220017170829
MSE 19: 30.35356103100823
MSE 20: 33.05474501674615
MSE 21: 28.892052227625182
MSE 22: 28.379694775084463
MSE 23: 25.893460320037544
MSE 24: 29.875673226168242
MSE 25: 30.556530609871576
MSE 26: 33.10747903373249
MSE 27: 26.423906955904172
MSE 28: 27.54286371002691
MSE 29: 33.518197933061224
MSE 30: 31.223382548606896
MSE 31: 27.443595096131357
MSE 32: 24.71300672636063
MSE 33: 25.28540840272379
MSE 34: 28.87301325875193
MSE 35: 29.559474266076936
MSE 36: 33.26661196958671
MSE 37: 24.3060546523159
MSE 38: 32.04165780274228
MSE 39: 28.